In [38]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid")

import fiona
import fiona.crs
import shapely
import rtree

import pandas as pd
import geopandas as gpd
import csv
import pyproj
import shapely.geometry as geom
import urllib2
import json
import numpy as np

In [2]:
subwayst = gpd.GeoDataFrame.from_file('subway routes/subwaystations.shp')

In [6]:
# MTA stations from GTFS
response = urllib2.urlopen("http://mtaapi.herokuapp.com/stations")
data = json.loads(response.read())    

In [12]:
stat_mta = pd.DataFrame(data['result'])
stat_mta['id'] = stat_mta['id'].apply(lambda x: x[:-1])
stat_mta.drop_duplicates(inplace=True)
stat_mta.index = range(len(stat_mta))

In [14]:
stat_mta.head()

,id,name
0,901,Grand Central - 42 St
1,902,Times Sq - 42 St
2,101,Van Cortlandt Park - 242 St
3,103,238 St
4,104,231 St


In [24]:
# Pull the locations
latitudes = []
longitudes = []
for i in stat_mta['id']:
    url1 = "http://mtaapi.herokuapp.com/stop?id="+str(i)+"S"
    url2 = "http://mtaapi.herokuapp.com/stop?id="+str(i)+"N"
    try:
        response = urllib2.urlopen(url1)
    except:
        response = urllib2.urlopen(url2)
    data = json.loads(response.read())
    try:
        latitudes.append(float(data['result']['lat']))
        longitudes.append(float(data['result']['lon']))
    except:
        latitudes.append(0)
        longitudes.append(0)

In [26]:
stat_mta['latitude']=latitudes
stat_mta['longitude']=longitudes

In [28]:
stat_mta.head()

,id,name,latitude,longitude
0,901,Grand Central - 42 St,40.752769,-73.979189
1,902,Times Sq - 42 St,40.755983,-73.986229
2,101,Van Cortlandt Park - 242 St,40.889248,-73.898583
3,103,238 St,40.884667,-73.900870
4,104,231 St,40.878856,-73.904834


In [152]:
stat_mta.to_csv("subway_station.csv",index=False)

In [33]:
stat_shp = pd.DataFrame({'id': subwayst['objectid'],
                        'name': subwayst['name'],
                        'latitude': subwayst['geometry'].apply(lambda geom: geom.y),
                         'longitude': subwayst['geometry'].apply(lambda geom: geom.x)
                        })

In [41]:
stat_shp.head()

,id,latitude,longitude,name
0,1.0,40.730054,-73.991070,Astor Pl
1,2.0,40.718803,-74.000193,Canal St
2,3.0,40.761728,-73.983849,50th St
3,4.0,40.680862,-73.974999,Bergen St
4,5.0,40.664714,-73.894886,Pennsylvania Ave


In [42]:
match = []
for i in stat_mta.index:
    dist=[]
    x1 = stat_mta.ix[i]['longitude']
    y1 = stat_mta.ix[i]['latitude']
    for j in stat_shp.index:
        x2 = stat_shp.ix[j]['longitude']
        y2 = stat_shp.ix[j]['latitude']
        dist.append(np.sqrt((x1-x2)**2+(y1-y2)**2))
    the_ix = np.where(dist==min(dist))
    choosen_station = stat_shp.ix[the_ix]['id']
    match.append(choosen_station)

In [53]:
match = [x.values[0] for x in match]

In [56]:
stat_shp.rename(columns={'id':'objectid'}, inplace=True)
stat_mta['objectid'] = match

In [60]:
merged_stat = pd.merge(stat_mta, stat_shp, how='left', on='objectid')

In [61]:
merged_stat.head()

,id,name_x,latitude_x,longitude_x,objectid,latitude_y,longitude_y,name_y
0,901,Grand Central - 42 St,40.752769,-73.979189,81.0,40.752769,-73.979189,Grand Central - 42nd St
1,902,Times Sq - 42 St,40.755983,-73.986229,147.0,40.755983,-73.986229,Times Sq - 42nd St
2,101,Van Cortlandt Park - 242 St,40.889248,-73.898583,271.0,40.889248,-73.898583,Van Cortlandt Park - 242nd St
3,103,238 St,40.884667,-73.900870,6.0,40.884667,-73.900870,238th St
4,104,231 St,40.878856,-73.904834,267.0,40.878856,-73.904834,231st St


In [79]:
the_dict = {}
for ix in merged_stat.index:
    the_dict[int(merged_stat.ix[ix]['objectid'])] = merged_stat.ix[ix]['id']

In [82]:
len(merged_stat['id'].unique())

493

# match with taxi

In [124]:
taxi = pd.read_csv('finaldf.csv', index_col=0)

In [125]:
taxi.head()

,Station,Taxi_Rides,Date,Hour
0,146,24,12,10
1,346,5,12,10
2,394,2,12,10
3,135,1,12,10
4,237,2,12,10


In [126]:
def tryconvert(x):
    try:
        return the_dict[x]
    except:
        return 0
converted = taxi['Station'].apply(lambda x: tryconvert(x))

In [127]:
len(np.unique(merged_stat['objectid']))

462

In [128]:
taxi['new_station'] = converted

In [129]:
taxi.head(10)

,Station,Taxi_Rides,Date,Hour,new_station
0,146,24,12,10,L06
1,346,5,12,10,B04
2,394,2,12,10,F21
3,135,1,12,10,250
4,237,2,12,10,G18
5,102,11,12,10,628
6,425,6,12,10,R26
7,7,12,12,10,A17
8,140,1,12,10,M14
9,37,3,12,10,G13


In [130]:
taxi=taxi.dropna()

In [131]:
taxi_L = taxi[taxi['new_station'].str.contains('L')==True].copy()
taxi_L['new_station'] =  taxi_L['new_station'].apply(lambda x: int(x[1:]))

In [132]:
taxi_L.head()

,Station,Taxi_Rides,Date,Hour,new_station
0,146,24,12,10,6
18,384,24,12,10,3
64,383,7,12,10,5
69,443,19,12,10,1
70,442,22,12,10,2


# merge

In [108]:
delay = pd.read_csv('delay_95.csv')

In [121]:
delay.head()

,keycol,delay
0,0_1_7,0
1,1_1_7,1
2,10_1_7,1
3,101_1_7,0
4,103_1_7,1


In [133]:
taxi['keycol'] = taxi.apply(lambda x: str(x.new_station)+'_'+str(x.Date)+'_'+str(x.Hour), axis=1)

In [134]:
taxi = taxi.drop(['Station','new_station','Date','Hour'], axis=1)
taxi.head()

,Taxi_Rides,keycol
0,24,L06_12_10
1,5,B04_12_10
2,2,F21_12_10
3,1,250_12_10
4,2,G18_12_10


In [135]:
taxi_L['keycol'] = taxi_L.apply(lambda x: str(x.new_station)+'_'+str(x.Date)+'_'+str(x.Hour), axis=1)

In [138]:
taxi_L = taxi_L.drop(['Station','new_station','Date','Hour'], axis=1)
taxi_L.head()

,Taxi_Rides,keycol
0,24,6_12_10
18,24,3_12_10
64,7,5_12_10
69,19,1_12_10
70,22,2_12_10


In [142]:
merged_L = pd.merge(delay, taxi_L, how='inner', on='keycol')
merged_L.head()

,keycol,delay,Taxi_Rides
0,1_1_7,1,27
1,2_1_7,0,16
2,3_1_7,1,16
3,5_1_7,3,4
4,6_1_7,2,35


In [144]:
merged_other = pd.merge(delay, taxi, how='inner', on='keycol')
merged_other.head()

,keycol,delay,Taxi_Rides
0,0_1_7,0,41
1,0_1_7,0,1
2,0_1_7,0,15
3,0_1_7,0,37
4,0_1_7,0,5


In [150]:
merged = pd.concat([merged_L,merged_other])
merged.index=range(len(merged))
merged.head()

,keycol,delay,Taxi_Rides
0,1_1_7,1,27
1,2_1_7,0,16
2,3_1_7,1,16
3,5_1_7,3,4
4,6_1_7,2,35


In [151]:
merged.to_csv('merged_4_30.csv', index=False)